In [0]:
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import os
import scipy.io as sio
from google.colab import drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_folder = '/content/drive/My Drive/DataInput/'
#test = sio.loadmat('/content/drive/My Drive/DataInput/test2.mat')
filenames = tf.gfile.ListDirectory(data_folder)

define a Model based on the ResNEt

In [0]:
flags = tf.app.flags
FLAGS = flags.FLAGS
#tensorboard
#keras
flags.FLAGS.train_dir = '/content/drive/My Drive/DataInput/'
class Model:
    """A neural network model.

    Currently only supports a feedforward architecture."""

    def __init__(self, name, features):
        self.name = name
        self.outputs = [features]

    def _get_layer_str(self, layer=None):
        if layer is None:
            layer = self.get_num_layers()

        return '%s_L%03d' % (self.name, layer + 1)

    def _get_num_inputs(self):
        return int(self.get_output().get_shape()[-1])

    def _glorot_initializer(self, prev_units, num_units, stddev_factor=1.0):
        """Initialization in the style of Glorot 2010.

        stddev_factor should be 1.0 for linear activations, and 2.0 for ReLUs"""
        stddev = np.sqrt(stddev_factor / np.sqrt(prev_units * num_units))
        return tf.truncated_normal([prev_units, num_units],
                                   mean=0.0, stddev=stddev)

    def _glorot_initializer_conv2d(self, prev_units, num_units, mapsize, stddev_factor=1.0):
        """Initialization in the style of Glorot 2010.

        stddev_factor should be 1.0 for linear activations, and 2.0 for ReLUs"""

        stddev = np.sqrt(stddev_factor / (np.sqrt(prev_units * num_units) * mapsize * mapsize))
        return tf.truncated_normal([mapsize, mapsize, prev_units, num_units],
                                   mean=0.0, stddev=stddev)

    def get_num_layers(self):
        return len(self.outputs)

    def add_batch_norm(self, scale=False):
        """Adds a batch normalization layer to this model.

        See ArXiv 1502.03167v3 for details."""

        # TBD: This appears to be very flaky, often raising InvalidArgumentError internally
        with tf.variable_scope(self._get_layer_str()):
            out = tf.contrib.layers.batch_norm(self.get_output(), scale=scale)

        self.outputs.append(out)
        return self

    def add_flatten(self):
        """Transforms the output of this network to a 1D tensor"""

        with tf.variable_scope(self._get_layer_str()):
            batch_size = int(self.get_output().get_shape()[0])
            out = tf.reshape(self.get_output(), [batch_size, -1])

        self.outputs.append(out)
        return self

    def add_dense(self, num_units, stddev_factor=1.0):
        """Adds a dense linear layer to this model.

        Uses Glorot 2010 initialization assuming linear activation."""

        assert len(self.get_output().get_shape()) == 2, "Previous layer must be 2-dimensional (batch, channels)"

        with tf.variable_scope(self._get_layer_str()):
            prev_units = self._get_num_inputs()

            # Weight term
            initw = self._glorot_initializer(prev_units, num_units,
                                             stddev_factor=stddev_factor)
            weight = tf.get_variable('weight', initializer=initw)

            # Bias term
            initb = tf.constant(0.0, shape=[num_units])
            bias = tf.get_variable('bias', initializer=initb)

            # Output of this layer
            out = tf.matmul(self.get_output(), weight) + bias

        self.outputs.append(out)
        return self

    def add_sigmoid(self):
        """Adds a sigmoid (0,1) activation function layer to this model."""

        with tf.variable_scope(self._get_layer_str()):
            prev_units = self._get_num_inputs()
            out = tf.nn.sigmoid(self.get_output())

        self.outputs.append(out)
        return self

    def add_softmax(self):
        """Adds a softmax operation to this model"""

        with tf.variable_scope(self._get_layer_str()):
            this_input = tf.square(self.get_output())
            reduction_indices = list(range(1, len(this_input.get_shape())))
            acc = tf.reduce_sum(this_input, reduction_indices=reduction_indices, keep_dims=True)
            out = this_input / (acc + FLAGS.epsilon)
            # out = tf.verify_tensor_all_finite(out, "add_softmax failed; is sum equal to zero?")

        self.outputs.append(out)
        return self

    def add_relu(self):
        """Adds a ReLU activation function to this model"""

        with tf.variable_scope(self._get_layer_str()):
            out = tf.nn.relu(self.get_output())

        self.outputs.append(out)
        return self

    def add_elu(self):
        """Adds a ELU activation function to this model"""

        with tf.variable_scope(self._get_layer_str()):
            out = tf.nn.elu(self.get_output())

        self.outputs.append(out)
        return self

    def add_lrelu(self, leak=.2):
        """Adds a leaky ReLU (LReLU) activation function to this model"""

        with tf.variable_scope(self._get_layer_str()):
            t1 = .5 * (1 + leak)
            t2 = .5 * (1 - leak)
            out = t1 * self.get_output() + \
                  t2 * tf.abs(self.get_output())

        self.outputs.append(out)
        return self

    def add_conv2d(self, num_units, mapsize=1, stride=1, stddev_factor=1.0):
        """Adds a 2D convolutional layer."""

        assert len(
            self.get_output().get_shape()) == 4 and "Previous layer must be 4-dimensional (batch, width, height, channels)"

        with tf.variable_scope(self._get_layer_str()):
            prev_units = self._get_num_inputs()

            # Weight term and convolution
            initw = self._glorot_initializer_conv2d(prev_units, num_units,
                                                    mapsize,
                                                    stddev_factor=stddev_factor)
            weight = tf.get_variable('weight', initializer=initw)
            out = tf.nn.conv2d(self.get_output(), weight,
                               strides=[1, stride, stride, 1],
                               padding='SAME')

            # Bias term
            initb = tf.constant(0.0, shape=[num_units])
            bias = tf.get_variable('bias', initializer=initb)
            out = tf.nn.bias_add(out, bias)

        self.outputs.append(out)
        return self

    def add_conv2d_transpose(self, num_units, mapsize=1, stride=1, stddev_factor=1.0):
        """Adds a transposed 2D convolutional layer"""

        assert len(
            self.get_output().get_shape()) == 4 and "Previous layer must be 4-dimensional (batch, width, height, channels)"

        with tf.variable_scope(self._get_layer_str()):
            prev_units = self._get_num_inputs()

            # Weight term and convolution
            initw = self._glorot_initializer_conv2d(prev_units, num_units,
                                                    mapsize,
                                                    stddev_factor=stddev_factor)
            weight = tf.get_variable('weight', initializer=initw)
            weight = tf.transpose(weight, perm=[0, 1, 3, 2])
            prev_output = self.get_output()
            output_shape = [FLAGS.batch_size,
                            int(prev_output.get_shape()[1]) * stride,
                            int(prev_output.get_shape()[2]) * stride,
                            num_units]
            out = tf.nn.conv2d_transpose(self.get_output(), weight,
                                         output_shape=output_shape,
                                         strides=[1, stride, stride, 1],
                                         padding='SAME')

            # Bias term
            initb = tf.constant(0.0, shape=[num_units])
            bias = tf.get_variable('bias', initializer=initb)
            out = tf.nn.bias_add(out, bias)

        self.outputs.append(out)
        return self

    def add_residual_block(self, num_units, mapsize=3, num_layers=2, stddev_factor=1e-3):
        """Adds a residual block as per Arxiv 1512.03385, Figure 3"""

        assert len(
            self.get_output().get_shape()) == 4 and "Previous layer must be 4-dimensional (batch, width, height, channels)"

        # Add projection in series if needed prior to shortcut
        if num_units != int(self.get_output().get_shape()[3]):
            self.add_conv2d(num_units, mapsize=1, stride=1, stddev_factor=1.)

        bypass = self.get_output()

        # Residual block
        for _ in range(num_layers):
            self.add_batch_norm()
            self.add_relu()
            self.add_conv2d(num_units, mapsize=mapsize, stride=1, stddev_factor=stddev_factor)

        self.add_sum(bypass)

        return self

    def add_bottleneck_residual_block(self, num_units, mapsize=3, stride=1, transpose=False):
        """Adds a bottleneck residual block as per Arxiv 1512.03385, Figure 3"""

        assert len(
            self.get_output().get_shape()) == 4 and "Previous layer must be 4-dimensional (batch, width, height, channels)"

        # Add projection in series if needed prior to shortcut
        if num_units != int(self.get_output().get_shape()[3]) or stride != 1:
            ms = 1 if stride == 1 else mapsize
            # bypass.add_batch_norm() # TBD: Needed?
            if transpose:
                self.add_conv2d_transpose(num_units, mapsize=ms, stride=stride, stddev_factor=1.)
            else:
                self.add_conv2d(num_units, mapsize=ms, stride=stride, stddev_factor=1.)

        bypass = self.get_output()

        # Bottleneck residual block
        self.add_batch_norm()
        self.add_relu()
        self.add_conv2d(num_units // 4, mapsize=1, stride=1, stddev_factor=2.)

        self.add_batch_norm()
        self.add_relu()
        if transpose:
            self.add_conv2d_transpose(num_units // 4,
                                      mapsize=mapsize,
                                      stride=1,
                                      stddev_factor=2.)
        else:
            self.add_conv2d(num_units // 4,
                            mapsize=mapsize,
                            stride=1,
                            stddev_factor=2.)

        self.add_batch_norm()
        self.add_relu()
        self.add_conv2d(num_units, mapsize=1, stride=1, stddev_factor=2.)

        self.add_sum(bypass)

        return self

    def add_sum(self, term):
        """Adds a layer that sums the top layer with the given term"""

        with tf.variable_scope(self._get_layer_str()):
            prev_shape = self.get_output().get_shape()
            term_shape = term.get_shape()
            # print("%s %s" % (prev_shape, term_shape))
            assert prev_shape == term_shape and "Can't sum terms with a different size"
            out = tf.add(self.get_output(), term)

        self.outputs.append(out)
        return self

    def add_mean(self):
        """Adds a layer that averages the inputs from the previous layer"""

        with tf.variable_scope(self._get_layer_str()):
            prev_shape = self.get_output().get_shape()
            reduction_indices = list(range(len(prev_shape)))
            assert len(reduction_indices) > 2 and "Can't average a (batch, activation) tensor"
            reduction_indices = reduction_indices[1:-1]
            out = tf.reduce_mean(self.get_output(), reduction_indices=reduction_indices)

        self.outputs.append(out)
        return self

    def add_upscale(self, size=None):
        """Adds a layer that upscales the output by 2x through nearest neighbor interpolation"""

        prev_shape = self.get_output().get_shape()
        if size is None:
            size = [2 * int(s) for s in prev_shape[1:3]]
        out = tf.image.resize_nearest_neighbor(self.get_output(), size)

        self.outputs.append(out)
        return self

    def add_concat(self, layer_add):
        last_layer = self.get_output()
        prev_shape = last_layer.get_shape()
        try:
            out = tf.concat(axis=3, values=[last_layer, layer_add])
            self.outputs.append(out)
        except:
            print('fail to concat {0} and {1}'.format(last_layer, layer_add))
        return self

    def add_layer(self, layer_add):
        self.outputs.append(layer_add)
        return self

    def get_output(self):
        """Returns the output from the topmost layer of the network"""
        return self.outputs[-1]

    def get_variable(self, layer, name):
        """Returns a variable given its layer and name.

        The variable must already exist."""

        scope = self._get_layer_str(layer)
        collection = tf.get_collection(tf.GraphKeys.VARIABLES, scope=scope)

        # TBD: Ugly!
        for var in collection:
            if var.name[:-2] == scope + '/' + name:
                return var

        return None

    def get_all_layer_variables(self, layer):
        """Returns all variables in the given layer"""
        scope = self._get_layer_str(layer)
        return tf.get_collection(tf.GraphKeys.VARIABLES, scope=scope)

UnrecognizedFlagError: ignored

In [0]:
# the first function of the model

In [0]:
def setup_inputs(sess, data_file_names, dataset_num, capacity_factor=1, num_threads=1):
  # FLAGS??? the meaning of FLAGS. sess is the self object?
  nrow = FLAGS.sample_size 
  ncol = FLAGS.sampel_size
  channels = FLAGS.complex_channels *2
  
  #build arraies for data_input and data_output
  data_input =[]
  data_outpt =[]
  for file in data_file_names:
    buff = sio.loadmat(file)
    inputs = buff['ImgK_single_slide']
    outputs = buff['prjChamber']
    if data_input == []:
      data_input = np.array(inputs)
      data_output = np.array(output)
    else:
      data_input = np.concatenate((data_input, np.array(inputs)), axis =3)
      data_output = np.concatenate((data_input, no.array(outputs)),axis =3)
  print('datasize = ', data_input.shape)
  buff_name = FLAGS.dataset_train + '.tfrecords'
  writer = tf.python_io.TFRecrodWriter(buff_name)
  # wrap with tqdm for a progress bar
  idx = 0;
  while idx < dataset_num:
    feature_vec = np.reshape(data_input[:,:,:,idx],[nrow*ncol*channels])
    label_vec = np.reshape(data_output[:,:,:,idx],[nrow*ncol*channels])
    # construct the Example proto object
    # need to check how to construct the object in python. Why do we need the object here? 
    example = tf.train.Features(
        Features = tf.train.Features(
            features = {
                'label': tf.train.Feature(
                    float_list = tf.train.FloatList(value=label_vec.astype("float"))
                ),
                'image': tf.train.Feature(
                    float_list = tf.train.FloatList(value = feature_vec.astype("float"))
                ),
            }
        )
    )
    serialized = example.SerializeToString()
    writer,write(serialized)
    idx+=1
    label_vec, image_vec = read_and_decode_single_example(buff_name,[nrow,ncol,channels])
    feature = tf.reashpe(image_vec,[nrow,ncol,channels])
    label = tf.reshape(label_vec, [nrow,ncol,channels])

    print('size_input_image',feature.get_shape())
    print('size_output_image',lable.get_Shape())

    features, labels = tf. train.batch([feature,label],batch_size = FLAG.batch_size,
                                       num_threads = num_threads,
                                       capacity = capacity_facotr * FLAGS.batch_size,
                                       name = 'labels_and_features')
    #what's the use for sess? 
    tf.train.start_queue_runners(sess=sess)
    return features, labels


the second function for create_model

In [0]:
def create_model(sess, features,labels, test_input):
  rows = int(features.get_shape()[1])
  cols = int(features.get_shape()[2])
  channels = int(features.get_shape()[3])
  gene_minput = test_input
  
  # the gene is representing the generator. gene_output_1, genge_var_list, gene_layers_1
  function_generator = lambda x,y,z,w:generator_model(x,y,z,w,num_dc_layers=-1, layer_output_skip=1)
  with tf.varable_scope('gene') as scope:
    gene_output_1, gene_var_list, gene_layers_1 = function_generator(sess, features, labels, channels)
    scope.reuse_variables()

    gene_output_2, _, gene_layers_2 = function_generator(sess,gene_output_1, labels, channels)
    scope.reuse_variables()

    gene_output = gene_output_1
    gene_output = tf.reshape(gene_output, [FLAGS.batch_size, rows, cols, channels])
    gene_layers = gene_layers_1

    # for testing input
    gene_moutput_1, _, gene_mlayers_1 = function_generator(sess, gene_minput, labels, channels)
    scope.reuse_variables()
    gene_moutput = gene_moutput_1
    
    gene_moutput = gene_moutput_1
    gene_moutput = tf.reshape(gene_moutput, [FLAGS.batch_size, rows, cols, channels])
    gene_mlayers = gene_mlayers_1

  disc_real_input = tf.identity(labels, name='disc_real_input')
  with tf.variable_scope('disc') as scope:
    disc_real_output, disc_var_list, disc_layers = discriminator_model(sess, features, disc_real_input, hybrid_disc = FLAGS.hybrid_disc)
    scope.reuse_variables()

    disc_fake_output,_,_ = discriminator_model(sess, features, gene_output, hybrid_disc= FLAGS.hybrid_disc)
  return [gene_minput, gene_moutput, gene_output, gnene_var_list, gene_layers, gene_malyers,
          disc_real_output, disc_fake_output, disc_moutput, disc_var_list, disc_layers, disc_mlayers]


In [0]:
def generator_model(sess, features, labels, channels, layer_output_skip =1, num_dc_layers = -1):
  mapsize = 3
  # is this the filter?
  res_units =[16,16,16]
  scale_changes = [0,0,0,0,0,0]
  print('use resnet without pooling:', res_units)
  old_vars = tf. global_variables()

  # check the reference arxiv 1603.05027
  model = Model('GEN', features)
  # loop different levels
  for ru in range(len(res_units)-1):
    nunits =res_units(ru)
    for j in range(2):
      model.add_residual_block(nunits, mapsize = mapsize)
      # spatial upscale 
      # and transposed convolution
      if scale_cahnges[ru] >0:
        model.add_upscale()

      model.add_batch_norm()
      model.add_relu()
      model.add_con2d_transpose(nunits, mapsize=mapsize)
    # finalization a la "all convolutional net"
    nunits = res_units[-1]
    model.add_conv2d(nunits, mapsize=mapsize, stride=1,stddev_factor=2.)
    model.add_relu()

    model.add_conv2d(nunits, mapsize=1, stride =1, stddev_factor=1.)
    
    new_vars = tf.global_variables()
    gene_vars = list(set(new_vars) - set(old_vars))

    # select subset of layers
    output_layers =[model.output[0]] + model.outputs[1:-1][::layer_output_skip] + [model.outputs[-1]]
    return model.get_output(), gene_vars, output_layers


In [0]:
def discriminator_model(sess, features, disc_input, layer_output_skip =1, hybrid_disc=0):
  # several important things: Model, three layers for Convolutional neural network, and one layer of convolutional neural network for. what's stddev? 
  mapsize =3
  layers =[8,16]
  old_vars = tf.global_variables()
  disc_hybrid =2 *disc_input -1
  print(hybrid_disc, 'discriminator input dimension: (0)'.format(disc_hybird.get_shape()))
  # why here has a Model?
  model = Model('DIS',disc_hybird)
  for layer in range(len(layers)):
    nunits = layers[layer]
    stddev_factor =2.0
    # why here adds an convolutional neural network? 
    model.add_conv2d(nunits, mapsize = mapsize, stride =2, stddev_factor = stddev_factor)
    model.add_batch_norm()
    model.add_relu()

  model.add_conv2d(nunits, mapsize=mapsize,stride=2, stddev_factor=stddev_factor)
  model.add_batch_norm()
  model.add_relu()

  model.add_conv2d(1,mapsize=1, stride =1, stddev_factor=stddev_factor)
  model.add_mean()
  new_vars = tf.global_variables()
  disc_vars = list(set(new_vars) - set(old_vars))
  output_layers = [model.output[0]]+ model.outputs[1:-1][::layer_output_skip] + [model.output[-1]]
  return model.get_output(), disc_vars, output_layers

The third function for generator loss function 

In [0]:
def create_generator_loss(disc_output, gene_output, features, labels):
  #using cross_entropy approach to evaluate the loss function 
  cross_entropy = tf.nn.sigmoid_cross_entropy_wth_logits(logits=disc_output, labels=tf.ones_like(disc_output))
  gene_ce_loss = tf.reduce_mean(cross_entropy, name='gene_ce_loss')
  # there are different types of loss function.Like LS cost, MSE loss, generator fool descriminator loss. gene_mse factor as a parameter
  # what is the tf.ones_like function? 
  #LS cost
  ls_loss = tf.square(disc_output - tf.ones_like(disc_output))
  gene_ls_loss =tf.reduce_mean(ls_loss,name ='gene_ls_loss')
  #mse loss
  gene_l1_loss = tf.reduce_mean(tf.abs(gene_output - labels), name ='gene_l1_loss')
  gene_l2_loss = tf.reduce_mean(tf.square(gene_output - lables), name = 'gene_l2_loss')
  gene_mse_loss = tf. add(FLAG.gene_l1l2_factor*gene_l1_loss, (1.01 - FLAGS.gene_l1l2_factor)*gene_l2_loss, name ='gene_mse_loss')
  
  #generator fool descriminator loss: gan LS or log loss
  gene_fool_loss =tf.add((1.0 - FLAGS.gene_log_factor)*gene_ls_loss, FLAGS.gene_log_factor * gene_ce_loss,name='gene_fool_loss')
  gene_mse_factor = FLAGS.gene_mse_factor

  ## gene_mse factor as a parameter
  gene_loss = tf.add((1.0 - gene_mse_factor)* gene_fool_loss, gene_mse_factor * gene_mse_loss, name ='gene_loss')

  # list of loss
  list_gene_losses = [gene_mse_loss, gene_l2_loss, gene_l1_loss, gene_fool_loss, gene_ls_loss, gene_ce_loss, gene_loss]
  return gene_loss, gene_mse_loss, gene_fool_loss, gene_ls_loss, list_gene_losses, gene_mse_factor


In [0]:
def create_optimizers(gene_loss, gene_var_list, disc_loss, disc_var_list):
  # about the AdamOptimizer. 
  global_step = tf.variable(0,dtype=tf.int64, trainable=False, name='global_step')
  learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
  gene_opti = tf.train.AdamOptimizer(learning_rate=learning_rate, betal=FLAGS.learning_betal,
                                     name='gene_optimizer')
  disc_opti = tf.train.AdamOptimizer(learning_rate = learning_rate, betal=FLAGS.learning_betal,name='disc_optimizer')

  gene_minimize = gene_opti.minimize(disc_loss, var_list =disc_var_list, name='disc_loss_minimize',global_step = global_step)
  disc_minimize = disc_opti.minimize(disc_loss, var_list=disc_var_list, name='disc_loss_minimize', global_step = global_step)
  return (global_step, learning_rate, gene_minimize, disc_minimize)
  #dict, tensor, placeholder,garph (operatin, parameter). 


Thr fourth function for discriminator loss function

In [0]:
def create_discriminator_loss(disc_real_output, disc_fake_output):
  #ls loss
  ls_loss_real = tf.square(disc_real_output - tf.ones_like(disc_real_output))
  disc_real_loss =tf.reduce_mean(ls_loss_real, name = 'disc_real_loss')
  
  ls_loss_fake = tf.square(disc_fake_output)
  disc_fake_loss = tf.reduce_mean(ls_loss_fake, name='disc_fake_loss')

  disc_loss =tf.add(disc_real_loss, disc_fake_loss, name='disc_loss')
  return disc_loss, disc_real_loss, disc_fake_loss

In [0]:
def setup_tensorflow(gpu_memory_fraction=0.9):
  # create session
  #config = tf.ConfigProto(log_device_placement = FLAGS.log_device_placement)
  config = tf.ConfigProto(log_device_placement = False)
  config.gpu_options.per_process_gpu_memory_fraction = min(gpu_memory_fraction, FLAGS.gpu_memory_fraction)
  sess = tf.Session(config=config)
  with sess.graph.as_default():
    tf.set_random_seed(FLAGS.random_seed)
  #random.seed(FLAGS.random_seed)
  #np.random.seed(FLAGS.random_seed)
  #np.randm.seed(FLAGS.random_seed)
  summary_writer = tf.summary.FileWriter(FLAGS.train_dir,sess.graph)
  return sess, summary_writer

In [0]:
def train():
  # set up the global tensorflow state
  #tf.app.flags.DEFINE_string('f', '', 'kernel')
  FLAGS.train_dir = '/content/drive/My Drive/DataInput/'
  sess, summary_writer = setup_tensorflow()
  # set up the train and test directories
  #prepare_dirs(delte_train_dir=True, shuffle_filename=False, separate_folder = True)
  data_folder = '/content/drive/My Drive/DataInput/'
  #test = sio.loadmat('/content/drive/My Drive/DataInput/test2.mat')
  #filenames = tf.gfile.ListDirectory(data_folder) 
  filenames = tf.gfile.ListDirectory(data_folder)
  filenames_input_train = [os.path.join(data_folder,f) for f in filenames if f.endswith('.mat')]
  #filenames = [os.path.join(data_folder,f) for f in filenames if f.endswith('.mat')]
  #filename_input_train = sio.loadmat(data_folder.join(test3.mat))

  #filenames_input_train = get_filenames(dir_file, shuffle_filename = True)
  print('train_filenames_input', filenames_input_train)

  # the first function for the train model
  #train_features, train_lables = setup_inputs(sess, filenames_input_train, FLAGS.sample_train)
  train_features, train_lables = setup_inputs(sess, data_file_names = filenames_input_train,dataset_num = 100)

  print('train_features_queue', train_features.get_shape())
  print('train_labels_queue', train_labels.get_shape())

  filenames_input_test = get_filenames(dir_file-FLAGS.dataset)

  # the second function for creating and initializing model
  [gene_minput,gene_moutput, gene_output, gene_var_list, gene_layers,\
   disc_real_output, disc_fake_output, disc_moutput, disc_var_list, disc_layers, disc_mlayers] =\
   srez_model.create_model(sess, train_features, train_lables, test_features)

  # the third function for generator loss function
  gene_loss, gene_mse_loss, gene_fool_loss, gene_ls_loss, list_gene_losses, gene_mse_factor =\
  srez_model.create_generator_loss(disc_fake_output, gene_output, train_features, train_labels)

  #the fourth function for discriminator loss function
  disc_loss, disc_real_loss, disc_fake_loss = srez_model.create_discriminator_loss(disc_real_output, disc_fake_output)

  (global_step, learning_rate, gene_minimize, disc_minimize) = srez_model.create_optimizer(gene_loss, gene_var_list, disc_loss, disc_var_list)

  # train model
  train_data = TrainData(locals())
  srez_train.train_model(train_data, FLAGS.sample_train, FLAGS.sample_test)


In [0]:
train()

UnrecognizedFlagError: ignored

In [0]:
def main(argv=None)
  if FLAGS.run == 'train':
    train()
if __name__ == '__main__'
  tf.app.run()

SyntaxError: ignored